In [1]:
# 항상 첫 번째 셀
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [1]:
import os
import csv
import json
import time
import re
from curl_cffi import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup

# SQLAlchemy
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError


# ==========================================
# 환경 변수 및 RDSClient 설정
# ==========================================
load_dotenv()

class RDSClient:
    def __init__(self):
        db_user = os.getenv('DB_USER')
        db_password = os.getenv('DB_PASSWORD')
        db_host = os.getenv('DB_HOST')
        db_port = os.getenv('DB_PORT', 3306)
        db_name = os.getenv('DB_NAME')

        self.db_url = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}?charset=utf8mb4"

        try:
            self.engine = create_engine(
                self.db_url,
                pool_size=5,
                max_overflow=10,
                pool_recycle=3600,
                echo=False
            )
            print("✅ DB Engine (Pool) 생성 완료")
        except Exception as e:
            print(f"❌ Engine 생성 중 오류 발생: {e}")
            self.engine = None

    def execute(self, query, params=None):
        if not self.engine:
            return None
        result_data = None
        try:
            with self.engine.connect() as connection:
                stmt = text(query)
                result = connection.execute(stmt, params or {})
                if result.returns_rows:
                    result_data = [dict(row) for row in result.mappings()]
                else:
                    connection.commit()
                    result_data = result.rowcount
        except SQLAlchemyError as e:
            print(f"⚠️ 쿼리 실행 에러: {e}")
            return None 
        return result_data

rds = RDSClient()

# ==========================================
# DB 컬럼 및 초기 설정
# ==========================================
cols_result = rds.execute("SHOW COLUMNS FROM product_bottom")
if not cols_result:
    print("❌ 테이블 정보를 가져오지 못했습니다.")
    exit()

db_cols = [row['Field'] for row in cols_result]
placeholders = ", ".join([f":{col}" for col in db_cols])
insert_sql = f"INSERT INTO product_bottom ({', '.join(db_cols)}) VALUES ({placeholders})"

# ==========================================
# CSV 로드
# ==========================================
goods_ids = []
try:
    with open("musinsa_bottom_ids.csv", newline="", encoding="utf-8") as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            goods_ids.append(row[0])
except FileNotFoundError:
    print("❌ CSV 파일 없음")
    exit()

total_count = len(goods_ids)

# ==========================================
# 시작 위치 설정
# ==========================================
# 이미 DB에 있는 개수가 아니라, 특정 인덱스부터 시작하고 싶다면 아래 숫자를 수정하세요.
# 현재 로직: DB에 있는 개수만큼 건너뛰고 시작
cnt_result = rds.execute("SELECT COUNT(*) as cnt FROM product_bottom")
saved_count = cnt_result[0]['cnt'] if cnt_result else 0
start_index = saved_count

print(f"전체: {total_count} / 저장됨: {saved_count} / 시작 인덱스: {start_index}")

# ==========================================
# 파싱 함수들
# ==========================================
def extract_int(text):
    return int(re.sub(r"[^0-9]", "", text)) if text else None

def extract_number(text):
    if text is None: return None
    if "만" in text:
        try: return str(int(float(text.replace("만", "")) * 10000))
        except: return text
    cleaned = re.sub(r"[^0-9]", "", text)
    return cleaned if cleaned != "" else ""

def clean_product_name(name):
    return re.sub(r"-.*", "", name).strip()

## ... (상단 import 및 설정 코드는 동일) ...

# ==========================================
# 크롤링 시작 (curl_cffi 사용)
# ==========================================
print("🚀 고속 크롤링 시작 (curl_cffi - 봇 탐지 진단 추가)")

session = requests.Session(impersonate="chrome") 
session.headers.update({
    "Referer": "https://www.musinsa.com/"
})

for idx in range(start_index, total_count):
    gid = goods_ids[idx]
    
    print(f"[{idx+1}/{total_count}] {gid} 요청 중...")

    try:
        url = f"https://www.musinsa.com/products/{gid}"
        
        # curl_cffi로 요청
        response = session.get(url, timeout=10)
        
        # 1. HTTP 상태 코드가 200이 아닌 경우
        if response.status_code != 200:
            print(f"⚠️ HTTP 요청 실패: {response.status_code}")
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        
        # ============================================================
        # [진단 로직 추가] 페이지 제목과 키워드로 봇 차단 확인
        # ============================================================
        page_title = soup.title.get_text(strip=True) if soup.title else "제목 없음"
        current_url = response.url # 리다이렉트 되었는지 확인
        
        # Cloudflare나 보안 솔루션이 내보내는 대표적인 차단 메시지들
        block_keywords = [
            "Access Denied", "Just a moment", "Security Check", 
            "Cloudflare", "Attention Required", "One more step", 
            "비정상적인 접근", "Shield"
        ]

        # 2. 치명적인 봇 차단 감지 (즉시 종료 권장)
        if any(keyword in page_title for keyword in block_keywords):
            print(f"\n🚨 [CRITICAL] 봇 탐지 시스템에 걸렸습니다! ({gid})")
            print(f"   - 페이지 제목: {page_title}")
            print(f"   - 감지된 키워드 포함. IP가 차단되었을 가능성이 높습니다.")
            print("   - 스크립트를 강제 종료합니다.")
            break  # 차단된 상태로 계속 요청하면 IP 영구 밴 위험이 있으므로 loop 탈출

        # 3. 상품명 파싱 시도
        name_tag = soup.select_one("div[class*='GoodsName__Wrap'] span")
        product_name = clean_product_name(name_tag.get_text(strip=True)) if name_tag else None

        # 4. 상품명 파싱 실패 시 원인 상세 분석
        if not product_name:
            print(f"⚠️ 상품 정보 파싱 실패 ({gid})")
            print(f"   - 페이지 제목: {page_title}")
            print(f"   - 최종 URL: {current_url}")

            if "Musinsa" == page_title or current_url.rstrip("/") == "https://www.musinsa.com":
                print("   -> [원인] 메인 페이지로 리다이렉트됨 (상품 삭제/품절/비공개)")
            elif "Login" in current_url or "login" in current_url:
                 print("   -> [원인] 로그인 필요 (성인 전용 상품 등)")
            else:
                print("   -> [원인] 알 수 없는 페이지 구조 (HTML 변경 확인 필요)")
            
            # 실패했으므로 다음 루프로 넘어감
            continue

        # ============================================================
        # 이하 기존 파싱 로직 동일
        # ============================================================

        # 2. 브랜드
        brand_tag = soup.select_one("div[class*='Brand__Wrap'] span[class*='BrandName']")
        brand = re.sub(r"[0-9\.]+만", "", brand_tag.get_text(strip=True)).strip() if brand_tag else None

        # 3~5 가격 정보
        normal_price = extract_int(soup.select_one("span.line-through").get_text()) if soup.select_one("span.line-through") else None
        sale_price_tag = soup.select_one("span[class*='CalculatedPrice']")
        sale_price = extract_int(sale_price_tag.get_text()) if sale_price_tag else normal_price 
        
        discount_tag = soup.select_one("span[class*='DiscountRate']")
        discount = extract_int(discount_tag.get_text()) if discount_tag else 0

        # 6 카테고리
        cats = soup.select("div[class*='Category__Wrap'] a")
        upper_category = cats[0].get_text(strip=True) if len(cats) > 0 else None
        lower_category = cats[1].get_text(strip=True) if len(cats) > 1 else None

        # 7 성별 & 누적판매
        gender = 0
        cumulative = ""
        layout_boxes = soup.select("dl[class*='Layout__Wrap'] div[class*='Layout__Box']")
        
        for box in layout_boxes:
            dt = box.find("dt")
            dd = box.find("dd")
            if dt and dd:
                label = dt.get_text(strip=True)
                val = dd.get_text(strip=True)
                
                if "성별" in label:
                    gender_map = {"남": 1, "여": 2, "공용": 0, "남녀공용": 0}
                    gender = gender_map.get(val, 0)
                elif "누적판매" in label:
                    cumulative = val

        # 8 별점
        rating_tag = soup.select_one("div[class*='Review__Wrap'] span")
        rating = float(rating_tag.get_text(strip=True)) if rating_tag else 0.0

        # 9 후기수
        review_cnt_tag = soup.select_one("div[class*='Review__Wrap'] span:nth-of-type(2)")
        review_cnt = int(extract_number(review_cnt_tag.get_text())) if review_cnt_tag else 0

        # 10 관심수
        like_tag = soup.select_one("div[class*='Like__Container'] span")
        like_cnt = extract_number(like_tag.get_text()) if like_tag else "0"

        # 11 스타일 태그
        tag_items = soup.select("ul[class*='ProductTags__List'] span")
        styles = ",".join([t.get_text(strip=True) for t in tag_items])

        # 12, 13 JSON 데이터
        size_json = "[]"
        fit_json = json.dumps({"핏": [], "계절감": []}, ensure_ascii=False)

        # DB INSERT
        values = (
            gid, product_name, brand, normal_price, sale_price,
            upper_category, lower_category, gender, rating,
            like_cnt, review_cnt, size_json, discount,
            fit_json, cumulative, styles
        )

        param_dict = dict(zip(db_cols, values))
        rds.execute(insert_sql, param_dict)

        # 성공 시 약간의 대기
        time.sleep(0.5)

    except Exception as e:
        print(f"❌ 오류 발생 ({gid}): {e}")
        with open("error_log.txt", "a", encoding="utf-8") as log:
            log.write(f"{gid}, {e}\n")

print("모든 작업 완료!")
# rds.engine.dispose()

✅ DB Engine (Pool) 생성 완료
전체: 59914 / 저장됨: 3604 / 시작 인덱스: 3604
🚀 고속 크롤링 시작 (curl_cffi - 봇 탐지 진단 추가)
[3605/59914] 4967542 요청 중...
⚠️ 상품 정보 파싱 실패 (4967542)
   - 페이지 제목: 바운더리(BOUNDARY) 원턱 스트링 롱 스웨트 팬츠 [브라운] - 사이즈 & 후기 | 무신사
   - 최종 URL: https://www.musinsa.com/products/4967542
   -> [원인] 알 수 없는 페이지 구조 (HTML 변경 확인 필요)
[3606/59914] 4466602 요청 중...
⚠️ 상품 정보 파싱 실패 (4466602)
   - 페이지 제목: 텍스(TEXX) TX101 세미 와이드 데님(생지) - 사이즈 & 후기 | 무신사
   - 최종 URL: https://www.musinsa.com/products/4466602
   -> [원인] 알 수 없는 페이지 구조 (HTML 변경 확인 필요)
[3607/59914] 3757155 요청 중...
⚠️ 상품 정보 파싱 실패 (3757155)
   - 페이지 제목: 무신사 스탠다드(MUSINSA STANDARD) 백 사틴 오피서 팬츠 [블루 그레이] - 사이즈 & 후기 | 무신사
   - 최종 URL: https://www.musinsa.com/products/3757155
   -> [원인] 알 수 없는 페이지 구조 (HTML 변경 확인 필요)
[3608/59914] 3539216 요청 중...
⚠️ 상품 정보 파싱 실패 (3539216)
   - 페이지 제목: 딥센스(DEEP SENSE) 워싱 벌룬 데님 D611 - 샌드엘로우 - 사이즈 & 후기 | 무신사
   - 최종 URL: https://www.musinsa.com/products/3539216
   -> [원인] 알 수 없는 페이지 구조 (HTML 변경 확인 필요)
[3609/59914] 3107548 요청 중...
⚠️ 상

KeyboardInterrupt: 